In [1]:
data_dir = '.'

import os
import math
import imgaug
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow as tf

import image_ocr

In [ ]:
# dataset


In [ ]:
train, validation = sklearn.model_selection.train_test_split(
    dataset, train_size=0.8, random_state=42
)
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.Affine(
      scale=(1.0, 1.2),
      rotate=(-5, 5)
    ),
    imgaug.augmenters.GaussianBlur(sigma=(0, 0.5)),
    imgaug.augmenters.Multiply((0.8, 1.2), per_channel=0.2)
])
generator_kwargs = {'width': 640, 'height': 640}
training_image_generator = image_ocr.datasets.get_detector_image_generator(
    labels=train,
    augmenter=augmenter,
    **generator_kwargs
)
validation_image_generator = image_ocr.datasets.get_detector_image_generator(
    labels=validation,
    **generator_kwargs
)

# Display an example
image, lines, confidence = next(training_image_generator)
canvas = image_ocr.tools.drawBoxes(image=image, boxes=lines, boxes_format='lines')
plt.imshow(canvas)

In [ ]:
# Initalized with default weights
detector = image_ocr.detection.Detector()

In [ ]:
batch_size = 1
training_generator, validation_generator = [
    detector.get_batch_generator(
        image_generator=image_generator, batch_size=batch_size
    ) for image_generator in
    [training_image_generator, validation_image_generator]
]

In [ ]:
detector.model.fit(
    training_generator,
    steps_per_epoch=math.ceil(len(train) / batch_size),
    epochs=1000,
    workers=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5),
        tf.keras.callbacks.CSVLogger(os.path.join(data_dir, 'detector_icdar2013.csv')),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(data_dir, 'detector_icdar2013.h5'))
    ],
    validation_data=validation_generator,
    validation_steps=math.ceil(len(validation) / batch_size)
)

## code of chatgpt after edits


In [13]:
import os
import math
import imgaug
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow as tf
import xml.etree.ElementTree as ET
from PIL import Image
import image_ocr

In [12]:
# physical_devices = tf.config.list_physical_devices('GPU')
# if physical_devices:
#     try:
#         tf.config.experimental.set_memory_growth(physical_devices[0], True)
#         print("GPU is available and ready!")
#     except RuntimeError as e:
#         print(e)

In [21]:
# Load Custom Dataset
class CustomDataset:
    def __init__(self, image_dir, annotations_dir):
        self.image_dir = image_dir
        self.annotations_dir = annotations_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def parse_annotation(self, annotation_file):
        tree = ET.parse(annotation_file)
        root = tree.getroot()
        lines = []

        for obj in root.findall('object'):
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
            lines.append([xmin, ymin, xmax, ymax])

        return lines

    def load_data(self):
        dataset = []
        for img_file in self.image_files:
            img_path = os.path.join(self.image_dir, img_file)
            annotation_path = os.path.join(self.annotations_dir, os.path.splitext(img_file)[0] + '.xml')
            if os.path.exists(annotation_path):
                boxes = self.parse_annotation(annotation_path)
                # Adding dummy confidence scores for each box
                confidence = [1.0] * len(boxes)  # Assuming a confidence of 1.0 for all annotations
                dataset.append((img_path, boxes, confidence))
        return dataset



In [22]:
# Define paths
image_dir = "F://Udaykiran//silent//ANPR-and-ATCC-for-Smart-Traffic-Management//dataset//indian_license_dataset//images"
annotations_dir = "F://Udaykiran//silent//ANPR-and-ATCC-for-Smart-Traffic-Management//dataset//indian_license_dataset//annote"
# Load dataset
custom_dataset = CustomDataset(image_dir, annotations_dir)
dataset = custom_dataset.load_data()

# Split data
train, validation = sklearn.model_selection.train_test_split(dataset, train_size=0.8, random_state=42)


In [23]:
# Augmentations
augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.Affine(scale=(1.0, 1.2), rotate=(-5, 5)),
    imgaug.augmenters.GaussianBlur(sigma=(0, 0.5)),
    imgaug.augmenters.Multiply((0.8, 1.2), per_channel=0.2)
])

# Generator arguments
generator_kwargs = {'width': 640, 'height': 640}

# Training and validation generators
training_image_generator = image_ocr.datasets.get_detector_image_generator(
    labels=train,
    augmenter=augmenter,
    **generator_kwargs
)
validation_image_generator = image_ocr.datasets.get_detector_image_generator(
    labels=validation,
    **generator_kwargs
)


In [24]:
# Display an example
image, lines, confidence = next(training_image_generator)
canvas = image_ocr.tools.drawBoxes(image=image, boxes=lines, boxes_format='lines')
plt.imshow(canvas)
plt.show()

# Initialize Detector
detector = image_ocr.detection.Detector()

# Batch size
batch_size = 1

# Batch Generators
training_generator, validation_generator = [
    detector.get_batch_generator(
        image_generator=image_generator, batch_size=batch_size
    ) for image_generator in [training_image_generator, validation_image_generator]
]

# Model Training
detector.model.fit(
    training_generator,
    steps_per_epoch=math.ceil(len(train) / batch_size),
    epochs=1000,
    workers=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5),
        tf.keras.callbacks.CSVLogger(os.path.join('.', 'detector_training.csv')),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join('.', 'detector_best_model.h5'))
    ],
    validation_data=validation_generator,
    validation_steps=math.ceil(len(validation) / batch_size)
)

TypeError: cannot unpack non-iterable int object

In [10]:
import tensorflow as tf
print(tf.__version__)


2.8.0
